# Langchain Agent with llms.txt (동기 버전)

이 노트북은 **Langchain**과 **Databricks**를 활용하여 llms.txt 기반 Tool-calling Agent를 구현하는 예제입니다.

## 동기(Sync) 버전이란?

> ✅ **Langchain 동기 방식의 표준 예제입니다.** Langchain의 `stream_mode="messages"`를 사용하여 동기 방식으로 토큰 단위 스트리밍을 구현합니다. 비동기 처리가 필요 없는 환경에서 더 간단하게 사용할 수 있습니다.

## llms.txt 접근 방식

이 노트북은 MCP 대신 **llms.txt** 파일을 활용하는 경량 접근 방식을 사용합니다:
- MCP보다 가볍고 설정이 간단
- RAG보다 정확한 문서 참조
- llms.txt를 인덱스로 사용하여 필요한 문서를 동적으로 가져옴

## 이 노트북에서 다루는 내용

- Langchain + Databricks Foundation Model API를 활용한 Agent 구현
- llms.txt 기반 문서 검색 및 페칭 도구 구현
- MLflow의 `ResponsesAgent`를 사용한 래퍼 클래스 작성
- Agent 출력 테스트 및 동기 스트리밍 처리
- Mosaic AI Agent Evaluation을 통한 평가
- Unity Catalog에 모델 등록 및 배포

## 실행 환경

- Databricks Serverless 또는 DBR 17 이상 클러스터
- Unity Catalog 활성화 필수

## 사전 요구사항

- 이 노트북의 모든 `TODO` 항목을 확인하고 수정하세요.

In [0]:
%pip install -U -qqqq backoff databricks-langchain uv databricks-agents mlflow-skinny[databricks]
dbutils.library.restartPython()

## Agent 코드 정의

아래에서 Agent 코드를 단일 셀에 정의합니다. `%%writefile` 매직 명령어를 사용하여 로컬 Python 파일로 저장하면, 이후 MLflow 로깅 및 배포 시 활용할 수 있습니다.

### 주요 구성 요소
- **LLMSTxtLoader**: llms.txt 파일 로드 및 인덱스 파싱
- **DocumentFetcher**: URL에서 문서 내용을 가져오는 유틸리티
- **Tool 정의**: search_docs, fetch_document, get_doc_index
- **ChatDatabricks**: Databricks Foundation Model API 연동

Agent에 추가할 수 있는 도구 예제는 [공식 문서](https://docs.databricks.com/generative-ai/agent-framework/agent-tool.html)를 참고하세요.

In [0]:
%%writefile langchain_sync_agent.py

import httpx
from functools import lru_cache
from urllib.parse import urljoin

from databricks_langchain import ChatDatabricks
from langchain.agents import create_agent
from langchain_core.tools import tool


# ==============================================================================
# llms.txt 로더
# ==============================================================================


def load_llms_txt(url: str) -> str:
    """llms.txt 파일을 로드하여 원문 반환"""
    with httpx.Client(timeout=10) as client:
        response = client.get(url)
        response.raise_for_status()
        content = response.text
        return re.sub(r"^# .+", "# 빗썸 API 문서 인덱스", content, count=1)


# ==============================================================================
# 문서 fetcher 함수
# ==============================================================================


@lru_cache(maxsize=128)
def fetch_document_content(url: str, base_url: str = "") -> str:
    """URL에서 문서를 가져옴 (캐싱 포함)

    Args:
        url: 가져올 문서 URL
        base_url: 상대 URL 처리를 위한 기본 URL

    Returns:
        문서 내용
    """
    # URL 정규화
    if not url.startswith("http"):
        if base_url and not base_url.endswith("/"):
            base_url = base_url.rsplit("/", 1)[0] + "/"
        url = urljoin(base_url, url)

    with httpx.Client(timeout=10, follow_redirects=True) as client:
        response = client.get(url)
        response.raise_for_status()
        return response.text


def extract_links(content: str) -> list[dict[str, str]]:
    """마크다운 콘텐츠에서 링크 추출

    Args:
        content: 마크다운 문서 내용

    Returns:
        링크 목록 (title, url)
    """
    links = []
    pattern = r"\[([^\]]+)\]\(([^)]+)\)"
    for match in re.finditer(pattern, content):
        title, url = match.groups()
        if not url.startswith("#"):  # 내부 앵커 링크 제외
            links.append({"title": title, "url": url})
    return links


# ==============================================================================
# Tool 정의
# ==============================================================================


def create_llms_txt_tools(llms_txt_url: str):
    """llms.txt 기반 tools 생성"""

    # llms.txt 로드
    llms_txt_content = load_llms_txt(llms_txt_url)

    @tool
    def fetch_document(url: str) -> str:
        """URL에서 문서 내용을 가져옵니다.

        Args:
            url: 가져올 문서의 URL

        Returns:
            문서 내용 (마크다운 형식)
        """
        try:
            content = fetch_document_content(url, base_url=llms_txt_url)

            # 추가 링크 정보 제공
            links = extract_links(content)
            if links:
                content += f"\n\n---\n📎 이 문서의 링크 ({len(links)}개):\n"
                for link in links:
                    content += f"- [{link['title']}]({link['url']})\n"

            return content
        except Exception as e:
            return f"문서 가져오기 오류: {str(e)}"

    return [fetch_document], llms_txt_content


# ==============================================================================
# System Prompt 생성
# ==============================================================================


def create_system_prompt(llms_txt_content: str) -> str:
    """llms.txt 원문을 포함한 시스템 프롬프트 생성"""

    return f"""당신은 빗썸 API 가이드 전문가입니다.

## 역할
사용자의 질문에 맞는 문서를 찾아서 fetch_document tool로 가져온 뒤 정확한 답변을 제공합니다.

## 사용 가능한 문서 목록
{llms_txt_content}

## 답변 방법
1. 위 문서 목록에서 관련 URL을 찾습니다
2. fetch_document(url)로 문서를 가져옵니다
3. 문서 내용을 바탕으로 정확하게 답변합니다
4. 추가 링크가 필요하면 해당 문서도 가져옵니다

## 주의사항
- 문서에 없는 내용은 추측하지 마세요
- 필요 시 여러 문서를 참조하세요
- 링크 안의 링크도 확인이 필요할 수 있습니다
"""


# ==============================================================================
# Agent 생성
# ==============================================================================


def create_llms_txt_agent(
    llms_txt_url: str = "https://examlple/llms.txt",
    model_name: str = "databricks-gpt-5-2",
    system_prompt: str | None = None,
):
    """llms.txt 기반 Agent 생성

    Args:
        llms_txt_url: llms.txt 파일 URL
        model_name: 사용할 LLM 모델 이름
        system_prompt: 커스텀 시스템 프롬프트 (None이면 자동 생성)

    Returns:
        Compiled Agent
    """

    # 모델 설정
    model = ChatDatabricks(endpoint=model_name)

    # Tools 및 llms.txt 원문 로드
    tools, llms_txt_content = create_llms_txt_tools(llms_txt_url)

    # 시스템 프롬프트 생성
    final_prompt = system_prompt or create_system_prompt(llms_txt_content)

    # Agent 생성
    return create_agent(model=model, tools=tools, system_prompt=final_prompt)


@lru_cache(maxsize=1)
def get_default_llms_txt_agent():
    """기본 Agent 캐싱 (싱글톤)"""
    return create_llms_txt_agent()


In [0]:
%%writefile langchain_sync_wrapper_agent.py
"""
Langchain Agent Wrapper (동기 버전)

MLflow ResponsesAgent를 래핑하여 Langchain Agent를 
MLflow의 predict/predict_stream 인터페이스와 호환되도록 합니다.

이 모듈은 Langchain의 동기 방식 표준 예제입니다.
stream_mode="messages"를 사용하여 토큰 단위 스트리밍을 지원합니다.
"""
from typing import Any, Generator
import json

import mlflow

from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    to_chat_completions_input,
    output_to_responses_items_stream,
)

from langchain_core.messages.ai import AIMessageChunk
from langchain_core.messages.tool import ToolMessage

from langchain_sync_agent import create_llms_txt_agent
from uuid import uuid4


class LangchainSyncResponsesAgent(ResponsesAgent):
    """Langchain Agent - 동기 버전 (토큰 스트리밍 지원)"""

    def __init__(self):
        self.agent = create_llms_txt_agent()

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """전체 응답을 한번에 반환 (동기)"""
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done" or event.type == "error"
        ]
        return ResponsesAgentResponse(
            output=outputs,
            custom_outputs=request.custom_inputs
        )

    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """동기 스트리밍

        stream_mode=["updates", "messages"]를 사용해서:
        - "messages": 실시간 텍스트 토큰 스트리밍 (사용자 경험)
        - "updates": 완전한 메시지 (AIMessage with tool_calls, ToolMessage) (히스토리 저장)

        이렇게 하면 Playground 대화 히스토리에 완전한 AIMessage (tool_calls 포함)가 저장되어
        다음 턴에서 ToolMessage가 올바른 형식으로 처리됨
        """

        cc_msgs = to_chat_completions_input([i.model_dump() for i in request.input])

        for event in self.agent.stream(
            {"messages": cc_msgs},
            stream_mode=["updates", "messages"],
        ):
            try:
                mode = event[0]
                data = event[1]

                # ============================================================
                # "messages" 모드: 실시간 텍스트 델타 스트리밍만
                # ============================================================
                if mode == "messages":
                    msg, metadata = data  # messages 모드는 (msg, metadata) 튜플

                    if isinstance(msg, AIMessageChunk) and msg.content:
                        yield ResponsesAgentStreamEvent(
                            **self.create_text_delta(
                                delta=msg.content,
                                item_id=msg.id or str(uuid4())
                            ),
                        )

                # ============================================================
                # "updates" 모드: 완전한 tool call/result 메시지만
                # ============================================================
                elif mode == "updates":
                    # updates는 {'node_name': {'messages': [...]}} 형태
                    # 전체 messages 배열을 output_to_responses_items_stream()에 전달
                    for node_data in data.values():
                        if len(node_data.get("messages", [])) > 0:
                            yield from output_to_responses_items_stream(node_data["messages"])

            except Exception as e:
                print(f"❌ Error in stream handling: {e}")
                print(f"   mode: {mode if 'mode' in locals() else 'unknown'}")
                print(f"   data type: {type(data) if 'data' in locals() else 'unknown'}")


# ==============================================================================
# MLflow 모델 등록
# ==============================================================================

mlflow.langchain.autolog()
AGENT = LangchainSyncResponsesAgent()
mlflow.models.set_model(AGENT)

## Agent 테스트

Agent와 상호작용하여 출력을 테스트합니다. `ResponsesAgent` 내에서 메서드를 수동으로 추적했기 때문에, Agent가 수행하는 각 단계의 트레이스를 확인할 수 있습니다. Langchain을 통한 LLM 호출은 autologging에 의해 자동으로 추적됩니다.

아래의 예시 입력을 실제 도메인에 맞는 예시로 변경하여 테스트하세요.

In [0]:
dbutils.library.restartPython()

In [0]:
from langchain_sync_wrapper_agent import AGENT

# 단일 응답 테스트 (비스트리밍)
AGENT.predict(
    {
        "input": [{"role": "user", "content": "사용 가능한 API 목록을 알려주세요."}], 
        "custom_inputs": {"session_id": "test-session-123"}
    },
)

In [0]:
from langchain_sync_wrapper_agent import AGENT

# 스트리밍 응답 테스트 (상세 출력)
for chunk in AGENT.predict_stream(
    {
        "input": [{"role": "user", "content": "인증 방법을 알려주세요."}], 
        "custom_inputs": {"session_id": "test-session-123"}
    },
):
    if chunk.type != 'response.output_item.done':
        print("===============\n")
    print(f"{chunk=}\n")
    print(chunk.model_dump(exclude_none=True))

### Agent를 MLflow 모델로 로깅

배포 시 자동 인증 패스스루를 위해 Databricks 리소스를 지정합니다.

- **TODO**: Unity Catalog 함수가 [Vector Search 인덱스](https://docs.databricks.com/generative-ai/agent-framework/unstructured-retrieval-tools.html)를 쿼리하거나 [외부 함수](https://docs.databricks.com/generative-ai/agent-framework/external-connection-tools.html)를 활용하는 경우, 해당 Vector Search 인덱스와 UC 연결 객체를 리소스로 포함해야 합니다. 자세한 내용은 [공식 문서](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough)를 참고하세요.

Agent 코드를 Python 파일에서 로깅합니다. [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code) 참고.

In [0]:
# 배포 시 자동 인증 패스스루를 위한 Databricks 리소스 설정
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint, DatabricksUCConnection
from pkg_resources import get_distribution

# TODO: 환경에 맞게 아래 설정값을 수정하세요
LLM_ENDPOINT_NAME = "databricks-gpt-5-2"  # 사용할 LLM 엔드포인트

resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
]

# 입력 예시 (모델 시그니처 추론용)
input_example = {
    "input": [
        {
            "role": "user",
            "content": "사용 가능한 API 목록을 알려주세요."
        }
    ],
    "custom_inputs": {
        "session_id": "test-session-123"
    }
}

# MLflow에 Agent 로깅
with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="langchain_sync_agent",
        python_model="langchain_sync_wrapper_agent.py",
        input_example=input_example,
        pip_requirements=[
            "databricks-langchain",
            "backoff",
            f"databricks-connect=={get_distribution('databricks-connect').version}",
        ],
        code_paths=[
            "langchain_sync_agent.py",
        ],
        resources=resources,
    )

## [Agent Evaluation](https://docs.databricks.com/mlflow3/genai/eval-monitor)으로 Agent 평가

평가 데이터셋의 요청이나 예상 응답을 수정하고, Agent를 반복 개선하면서 평가를 실행할 수 있습니다. MLflow를 활용하여 계산된 품질 메트릭을 추적합니다.

[사전 정의된 LLM 스코어러](https://docs.databricks.com/mlflow3/genai/eval-monitor/predefined-judge-scorers)를 사용하거나, [커스텀 메트릭](https://docs.databricks.com/mlflow3/genai/eval-monitor/custom-scorers)을 추가하여 Agent를 평가하세요.

In [0]:
import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety, RetrievalRelevance, RetrievalGroundedness

# 평가 데이터셋 정의
# TODO: 실제 사용 사례에 맞게 질문과 예상 응답을 수정하세요
eval_dataset = [
    {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "사용 가능한 API 목록을 알려주세요."
                }
            ]
        },
        "expected_response": None
    },
    {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "인증 방법을 알려주세요."
                }
            ]
        },
        "expected_response": None
    }
]

# Agent 평가 실행
eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda input: AGENT.predict({"input": input, "custom_inputs": {"session_id": "evaluation-session"}}),
    scorers=[RelevanceToQuery(), Safety()],  # 필요에 따라 스코어러 추가
)

# MLflow UI에서 평가 결과 확인 (콘솔 출력 참조)

## 배포 전 Agent 검증

Agent를 등록하고 배포하기 전에 [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API를 통해 사전 배포 검증을 수행합니다. 자세한 내용은 [공식 문서](https://docs.databricks.com/machine-learning/model-serving/model-serving-debug.html#validate-inputs)를 참고하세요.

In [0]:
# 배포 전 검증 - 로깅된 모델이 정상적으로 예측을 수행하는지 확인
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/langchain_sync_agent",
    input_data={
        "input": [{"role": "user", "content": "사용 가능한 API 목록을 알려주세요."}], 
        "custom_inputs": {"session_id": "validation-session"}
    },
    env_manager="uv",
)

## Unity Catalog에 모델 등록

아래의 `catalog`, `schema`, `model_name`을 업데이트하여 MLflow 모델을 Unity Catalog에 등록합니다.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: Unity Catalog 모델의 catalog, schema, model_name을 환경에 맞게 수정하세요
catalog = "ml"
schema = "default"
model_name = "langchain-sync-agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# Unity Catalog에 모델 등록
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

## Agent 배포

Agent를 Databricks Model Serving 엔드포인트로 배포합니다.

In [0]:
from databricks import agents

# 참고: scale_to_zero=True를 전달하면 비용 절감을 위한 스케일 투 제로가 활성화됩니다.
# 프로덕션 워크로드에는 권장되지 않습니다 (스케일 제로 상태에서는 용량이 보장되지 않음).
# 스케일 제로 상태의 엔드포인트는 다시 스케일업하는 동안 응답 시간이 길어질 수 있습니다.
agents.deploy(
    UC_MODEL_NAME, 
    uc_registered_model_info.version, 
    tags={"endpointSource": "local_development"}
)

## 다음 단계

Agent가 배포되면 다음 작업을 수행할 수 있습니다:

- **AI Playground에서 테스트**: 추가 검증을 위해 대화형으로 Agent 테스트
- **피드백 수집**: 조직 내 전문가(SME)와 Agent 공유하여 피드백 수집
- **프로덕션 애플리케이션에 통합**: 실제 서비스에 Agent 임베딩

자세한 내용은 [공식 문서](https://docs.databricks.com/generative-ai/deploy-agent.html)를 참고하세요.